In [1]:
from pyspark.sql import SparkSession,DataFrame
import os
import numpy as np
from pyspark.sql import functions as f
from pyspark.sql.types import *
import io
import time
from pyspark.sql import Row

In [2]:
worker_image="inseefrlab/jupyter-datascience:py3.9.7-spark3.1.2"

local=False
if local:
    spark = SparkSession \
    .builder.master("local[4]") \
    .appName("SparkReadLargeDataSet") \
    .getOrCreate()
else: 
    spark = SparkSession \
    .builder.master("k8s://https://kubernetes.default.svc:443") \
    .appName("SparkReadLargeDataSet") \
    .config("spark.kubernetes.container.image", worker_image) \
    .config("spark.kubernetes.authenticate.driver.serviceAccountName", os.environ['KUBERNETES_SERVICE_ACCOUNT']) \
    .config("spark.kubernetes.driver.pod.name", os.environ['KUBERNETES_POD_NAME']) \
    .config("spark.executor.instances", "8") \
    .config("spark.executor.memory","8g") \
    .config("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE']) \
    .getOrCreate()

2021-10-25 15:48:30,440 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
spark.sparkContext.stop()

In [3]:
! kubectl get pods

I1025 15:49:01.239398    1183 request.go:665] Waited for 1.174309501s due to client-side throttling, not priority and fairness, request: GET:https://kubernetes.default/apis/authorization.k8s.io/v1beta1?timeout=32s
NAME                                               READY   STATUS      RESTARTS   AGE
amundsen-elasticsearch-master-0                    1/1     Running     0          5d5h
amundsen-elasticsearch-master-1                    1/1     Running     0          5d5h
amundsen-elasticsearch-master-2                    1/1     Running     0          5d5h
amundsen-test-frontend-5648d65454-ffgpt            1/1     Running     0          5d5h
amundsen-test-metadata-c57c47666-7bjsz             1/1     Running     0          5d5h
amundsen-test-search-5c8455d784-g56vh              1/1     Running     0          5d5h
datahub-datahub-frontend-6ff5494f44-7gcpx          1/1     Running     0          5d
datahub-datahub-gms-5b66978446-g2526               1/1     Running     0          5d
datahub-

In [4]:
data_path="s3a://pengfei/diffusion/data_format/ny_taxis/parquet/raw_2011_2012"

In [5]:
df=spark.read.parquet(data_path)
df.show()

+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+------------------+-----------------+----------------+------------+-----------+-----+-------+----------+------------+------------+
|vendor_id|          pickup_at|         dropoff_at|passenger_count|trip_distance|pickup_longitude|pickup_latitude|rate_code_id|store_and_fwd_flag|dropoff_longitude|dropoff_latitude|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|total_amount|
+---------+-------------------+-------------------+---------------+-------------+----------------+---------------+------------+------------------+-----------------+----------------+------------+-----------+-----+-------+----------+------------+------------+
|      VTS|2009-01-04 02:52:00|2009-01-04 03:02:00|              1|         2.63|       -73.99196|      40.721565|        null|              null|       -73.993805|       40.695923|        CASH|        8.9|  0.5|   null|      

In [6]:
df.count()

1547741381

In [7]:
df.rdd.getNumPartitions()

312

In [9]:
output_path="s3a://pengfei/diffusion/data_format/ny_taxis/csv/2009_2019"
df.write.option("header",True).csv(output_path)

2021-10-25 15:56:47,849 WARN commit.AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
2021-10-25 15:58:41,118 WARN scheduler.TaskSetManager: Lost task 36.0 in stage 4.0 (TID 351) (10.233.118.19 executor 6): java.lang.UnsupportedOperationException: org.apache.parquet.column.values.dictionary.PlainValuesDictionary$PlainBinaryDictionary
	at org.apache.parquet.column.Dictionary.decodeToInt(Dictionary.java:45)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetDictionary.decodeToInt(ParquetDictionary.java:31)
	at org.apache.spark.sql.execution.vectorized.OnHeapColumnVector.getInt(OnHeapColumnVector.java:298)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.s

Py4JJavaError: An error occurred while calling o61.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:231)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:188)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at org.apache.sp

2021-10-25 15:58:45,632 WARN scheduler.TaskSetManager: Lost task 31.0 in stage 4.0 (TID 346) (10.233.121.55 executor 1): TaskKilled (Stage cancelled)
2021-10-25 15:58:45,867 WARN scheduler.TaskSetManager: Lost task 33.0 in stage 4.0 (TID 348) (10.233.115.80 executor 5): TaskKilled (Stage cancelled)
